In [1]:
from optimum.intel import OVQuantizer
from transformers import AutoModelForCausalLM, AutoTokenizer
from os.path import join
import os

INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, onnx, openvino


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/home/user/anaconda3/envs/openvino_optimum/lib/python3.11/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [2]:
## login to huggingfacehub to get access to pretrained model 
from huggingface_hub import notebook_login, whoami

try:
    whoami()
    print('Authorization token already provided')
except OSError:
    notebook_login()


Authorization token already provided


In [3]:
model_name= "meta-llama/Llama-2-7b-hf"
model_dir="Llama-2-7b-hf"
#"HuggingFaceH4/zephyr-7b-beta"
#pt_model = AutoModelForCausalLM.from_pretrained(model_name)


In [4]:
pt_model = AutoModelForCausalLM.from_pretrained(model_name)
quantizer = OVQuantizer.from_pretrained(pt_model)
quantizer.quantize(save_directory=join(model_dir,"INT_8"), weights_only=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

No configuration describing the quantization process was provided, a default OVConfig will be generated.
Using framework PyTorch: 2.1.0+cpu
Overriding 1 configuration item(s)
	- use_cache -> True
/home/user/anaconda3/envs/openvino_optimum/lib/python3.11/site-packages/transformers/models/llama/modeling_llama.py:146: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if seq_len > self.max_seq_len_cached:
/home/user/anaconda3/envs/openvino_optimum/lib/python3.11/site-packages/transformers/models/llama/modeling_llama.py:375: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other 

In [15]:
tokenizer=AutoTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained(join(model_dir,'tokenizer'))

('Llama-2-7b-hf/tokenizer/tokenizer_config.json',
 'Llama-2-7b-hf/tokenizer/special_tokens_map.json',
 'Llama-2-7b-hf/tokenizer/tokenizer.model',
 'Llama-2-7b-hf/tokenizer/added_tokens.json',
 'Llama-2-7b-hf/tokenizer/tokenizer.json')

# loading

it is a good idea to first quantize and then restart the notebook for the loading. 

this is because the old model is still in memory and we want to avoid memory issues

In [1]:
from optimum.intel.openvino import OVModelForCausalLM
from transformers import AutoTokenizer
from os.path import join

INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, onnx, openvino


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/home/user/anaconda3/envs/openvino_optimum/lib/python3.11/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [2]:
#model_name= "meta-llama/Llama-2-7b-hf"
model_dir="Llama-2-7b-hf"

In [3]:
model=OVModelForCausalLM.from_pretrained(join(model_dir,"INT_8"))
model

Compiling the model to CPU ...
Setting OpenVINO CACHE_DIR to Llama-2-7b-hf/INT_8/model_cache


In [5]:
tokenizer=AutoTokenizer.from_pretrained(join(model_dir,"tokenizer"))
tokenizer

LlamaTokenizerFast(name_or_path='Llama-2-7b-hf/tokenizer', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [6]:
inputs=tokenizer(['hey I am input'])
outputs=model.generate(**inputs)
print(tok.batch_decode(outputs)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


AttributeError: 'list' object has no attribute 'shape'